# Data generation

In [ ]:
import pandas as pd
import numpy as np
import os
import sys
from PIL import Image
import imgaug.augmenters as iaa
 
from tools import read_image_from_url, plot_image

print("System version: {}".format(sys.version))
print("imgaug version: {}".format(imgaug.version))

In [ ]:
neko_url = "https://raw.githubusercontent.com/miguelgfierro/sciblog_support/master/Intro_to_Machine_Learning_API/neko.jpg"
neko = read_image_from_url(neko_url)
plot_image(neko)